## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('demo').getOrCreate()

In [0]:
# File location and type
file_location = "/FileStore/tables"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
casedf = spark.read.csv("/FileStore/tables/Case-4.csv",header=True,inferSchema=True)
casedf.show(5)

+--------+--------+------------+-----+--------------------+---------+---------+----------+
| case_id|province|        city|group|      infection_case|confirmed| latitude| longitude|
+--------+--------+------------+-----+--------------------+---------+---------+----------+
| 1000001|   Seoul|  Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
| 1000002|   Seoul|   Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
| 1000003|   Seoul|     Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
| 1000004|   Seoul|Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
| 1000005|   Seoul|   Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
+--------+--------+------------+-----+--------------------+---------+---------+----------+
only showing top 5 rows



In [0]:
casedf.printSchema()

root
 |--  case_id: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: boolean (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [0]:
casedf.count()

Out[25]: 174

In [0]:
regdf = spark.read.csv("/FileStore/tables/Region.csv",header=True,inferSchema=True)
regdf.show(5,truncate=False)

+-----+--------+-----------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|code |province|city       |latitude |longitude |elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+-----+--------+-----------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|10000|Seoul   |Seoul      |37.566953|126.977977|607                    |830               |48              |1.44         |15.38                   |5.8                |22739             |
|10010|Seoul   |Gangnam-gu |37.518421|127.047222|33                     |38                |0               |4.18         |13.17                   |4.3                |3088              |
|10020|Seoul   |Gangdong-gu|37.530492|127.123837|27         

In [0]:
regdf.printSchema()

root
 |-- code: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elementary_school_count: integer (nullable = true)
 |-- kindergarten_count: integer (nullable = true)
 |-- university_count: integer (nullable = true)
 |-- academy_ratio: double (nullable = true)
 |-- elderly_population_ratio: double (nullable = true)
 |-- elderly_alone_ratio: double (nullable = true)
 |-- nursing_home_count: integer (nullable = true)



In [0]:
regdf.count()

Out[28]: 244

In [0]:
casedf.describe()

Out[29]: DataFrame[summary: string,  case_id: string, province: string, city: string, infection_case: string, confirmed: string, latitude: string, longitude: string]

In [0]:
regdf.describe()

Out[30]: DataFrame[summary: string, code: string, province: string, city: string, latitude: string, longitude: string, elementary_school_count: string, kindergarten_count: string, university_count: string, academy_ratio: string, elderly_population_ratio: string, elderly_alone_ratio: string, nursing_home_count: string]

In [0]:
from pyspark.sql.functions import *
casedf.select([count(when(isnull(c), c)).alias(c) for c in casedf.columns]).show()

+--------+--------+----+-----+--------------+---------+--------+---------+
| case_id|province|city|group|infection_case|confirmed|latitude|longitude|
+--------+--------+----+-----+--------------+---------+--------+---------+
|       0|       0|   0|    0|             0|        0|       0|        0|
+--------+--------+----+-----+--------------+---------+--------+---------+



In [0]:
regdf.select([count(when(isnull(c), c)).alias(c) for c in regdf.columns]).show()
#No null values

+----+--------+----+--------+---------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|code|province|city|latitude|longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+----+--------+----+--------+---------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|   0|       0|   0|       0|        0|                      0|                 0|               0|            0|                       0|                  0|                 0|
+----+--------+----+--------+---------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+



In [0]:
casedf.limit(5).show()

+--------+--------+------------+-----+--------------------+---------+---------+----------+
| case_id|province|        city|group|      infection_case|confirmed| latitude| longitude|
+--------+--------+------------+-----+--------------------+---------+---------+----------+
| 1000001|   Seoul|  Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
| 1000002|   Seoul|   Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
| 1000003|   Seoul|     Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
| 1000004|   Seoul|Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
| 1000005|   Seoul|   Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
+--------+--------+------------+-----+--------------------+---------+---------+----------+



In [0]:
regdf.limit(5).show()

+-----+--------+-----------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
| code|province|       city| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+-----+--------+-----------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|10000|   Seoul|      Seoul|37.566953|126.977977|                    607|               830|              48|         1.44|                   15.38|                5.8|             22739|
|10010|   Seoul| Gangnam-gu|37.518421|127.047222|                     33|                38|               0|         4.18|                   13.17|                4.3|              3088|
|10020|   Seoul|Gangdong-gu|37.530492|127.123837|           

In [0]:
newdf = casedf.withColumnRenamed('province','New_Province')
newdf.show(5)

+--------+------------+------------+-----+--------------------+---------+---------+----------+
| case_id|New_Province|        city|group|      infection_case|confirmed| latitude| longitude|
+--------+------------+------------+-----+--------------------+---------+---------+----------+
| 1000001|       Seoul|  Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
| 1000002|       Seoul|   Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
| 1000003|       Seoul|     Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
| 1000004|       Seoul|Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
| 1000005|       Seoul|   Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
+--------+------------+------------+-----+--------------------+---------+---------+----------+
only showing top 5 rows



In [0]:
casedf.select(casedf.province,casedf.city).show(5)

+--------+------------+
|province|        city|
+--------+------------+
|   Seoul|  Yongsan-gu|
|   Seoul|   Gwanak-gu|
|   Seoul|     Guro-gu|
|   Seoul|Yangcheon-gu|
|   Seoul|   Dobong-gu|
+--------+------------+
only showing top 5 rows



In [0]:
regdf.select(regdf.code,regdf.province,regdf.city).show(5)

+-----+--------+-----------+
| code|province|       city|
+-----+--------+-----------+
|10000|   Seoul|      Seoul|
|10010|   Seoul| Gangnam-gu|
|10020|   Seoul|Gangdong-gu|
|10030|   Seoul| Gangbuk-gu|
|10040|   Seoul| Gangseo-gu|
+-----+--------+-----------+
only showing top 5 rows



In [0]:
casedf.na.fill('NA').show(5) 

+--------+--------+------------+-----+--------------------+---------+---------+----------+
| case_id|province|        city|group|      infection_case|confirmed| latitude| longitude|
+--------+--------+------------+-----+--------------------+---------+---------+----------+
| 1000001|   Seoul|  Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
| 1000002|   Seoul|   Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
| 1000003|   Seoul|     Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
| 1000004|   Seoul|Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
| 1000005|   Seoul|   Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
+--------+--------+------------+-----+--------------------+---------+---------+----------+
only showing top 5 rows



In [0]:
#For example, we may want to find out all the different
#infection_case in Daegu Province with more than 10 confirmed cases
casedf.filter((col("province") == "Daegu") & (col("confirmed") > 10))\
      .select("infection_case","province","confirmed").show()

+--------------------+--------+---------+
|      infection_case|province|confirmed|
+--------------------+--------+---------+
|  Shincheonji Church|   Daegu|     4511|
|Second Mi-Ju Hosp...|   Daegu|      196|
|Hansarang Convale...|   Daegu|      124|
|Daesil Convalesce...|   Daegu|      101|
|     Fatima Hospital|   Daegu|       39|
|     overseas inflow|   Daegu|       41|
|contact with patient|   Daegu|      917|
|                 etc|   Daegu|      747|
+--------------------+--------+---------+



In [0]:
casedf.sort('confirmed').select('confirmed').show(10)

+---------+
|confirmed|
+---------+
|        0|
|        0|
|        0|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
+---------+
only showing top 10 rows



In [0]:
casedf.sort(casedf.confirmed.desc()).select('confirmed').show(10)

+---------+
|confirmed|
+---------+
|     4511|
|      917|
|      747|
|      566|
|      305|
|      298|
|      196|
|      190|
|      162|
|      139|
+---------+
only showing top 10 rows



In [0]:
df1 = casedf.select(col("latitude").cast('double').alias("latitude"))
df1.printSchema()

root
 |-- latitude: double (nullable = true)



In [0]:
casedf.groupBy(["province"]).sum("confirmed").show()

+-----------------+--------------+
|         province|sum(confirmed)|
+-----------------+--------------+
|           Sejong|            49|
|            Ulsan|            51|
|Chungcheongbuk-do|            60|
|       Gangwon-do|            62|
|          Gwangju|            43|
| Gyeongsangbuk-do|          1324|
|            Daegu|          6680|
| Gyeongsangnam-do|           132|
|          Incheon|           202|
|          Jeju-do|            19|
|      Gyeonggi-do|          1000|
|            Busan|           156|
|          Daejeon|           131|
|            Seoul|          1280|
|Chungcheongnam-do|           158|
|     Jeollabuk-do|            23|
|     Jeollanam-do|            25|
+-----------------+--------------+



In [0]:
casedf.groupBy(["city"]).sum("confirmed").show()

+---------------+--------------+
|           city|sum(confirmed)|
+---------------+--------------+
|     Gangnam-gu|            18|
|     Cheonan-si|           103|
|from other city|          1217|
|      Anyang-si|            39|
|      Gwanak-gu|           149|
|     Yongsan-gu|           139|
|        Dong-gu|            44|
|         Sejong|            39|
|     Gangseo-gu|             0|
|       Wonju-si|             4|
|     Suyeong-gu|             5|
|   Geochang-gun|            18|
|  Dongdaemun-gu|            17|
|     Dongnae-gu|            39|
|         Jin-gu|             4|
|     Yangsan-si|             3|
|    Changwon-si|             7|
|         Nam-gu|          4511|
|   Gyeongsan-si|            99|
|      Jongno-gu|            17|
+---------------+--------------+
only showing top 20 rows



In [0]:
casedf.join(regdf,casedf.city==regdf.city,"inner")\
      .select(casedf.province,casedf.confirmed,regdf.code)\
      .show()

+--------+---------+-----+
|province|confirmed| code|
+--------+---------+-----+
|   Seoul|        4|10010|
|   Seoul|        1|10010|
|   Seoul|        6|10010|
|   Seoul|        7|10010|
|   Seoul|        0|10040|
|   Seoul|       30|10050|
|   Seoul|      119|10050|
|   Seoul|        3|10070|
|   Seoul|       41|10070|
|   Seoul|       95|10070|
|   Seoul|        6|10080|
|   Seoul|       43|10100|
|   Seoul|       17|10110|
|   Seoul|        5|10140|
|   Seoul|        5|10150|
|   Seoul|       13|10160|
|   Seoul|        3|10190|
|   Seoul|       43|10190|
|   Seoul|        3|10200|
|   Seoul|      139|10210|
+--------+---------+-----+
only showing top 20 rows



In [0]:
casedf.join(regdf,casedf.province==regdf.province,"inner")\
      .select(casedf.city,casedf.confirmed,regdf.code)\
      .show()

+---------------+---------+-----+
|           city|confirmed| code|
+---------------+---------+-----+
|              -|      100|10000|
|              -|      162|10000|
|              -|      298|10000|
|        Guro-gu|        3|10000|
|              -|        1|10000|
|from other city|        2|10000|
|        Jung-gu|        3|10000|
|from other city|        4|10000|
|     Gangseo-gu|        0|10000|
|     Gangnam-gu|        4|10000|
|from other city|        1|10000|
|      Seocho-gu|        5|10000|
|   Yangcheon-gu|        3|10000|
|     Gangnam-gu|        1|10000|
|Yeongdeungpo-gu|        3|10000|
|        Jung-gu|       13|10000|
|from other city|        5|10000|
|from other city|        8|10000|
|   Geumcheon-gu|        6|10000|
|from other city|        1|10000|
+---------------+---------+-----+
only showing top 20 rows



In [0]:
casedf.createOrReplaceTempView("case")

In [0]:
spark.sql("Select infection_case,province,confirmed from case\
          where province = 'Daegu' and confirmed > 10")\
          .show()

+--------------------+--------+---------+
|      infection_case|province|confirmed|
+--------------------+--------+---------+
|  Shincheonji Church|   Daegu|     4511|
|Second Mi-Ju Hosp...|   Daegu|      196|
|Hansarang Convale...|   Daegu|      124|
|Daesil Convalesce...|   Daegu|      101|
|     Fatima Hospital|   Daegu|       39|
|     overseas inflow|   Daegu|       41|
|contact with patient|   Daegu|      917|
|                 etc|   Daegu|      747|
+--------------------+--------+---------+



In [0]:
spark.sql("select province, infection_case,confirmed, dense_rank() over(partition by province order by confirmed desc)\
            as rank from case").show()

+-----------------+--------------------+---------+----+
|         province|      infection_case|confirmed|rank|
+-----------------+--------------------+---------+----+
|            Busan|       Onchun Church|       39|   1|
|            Busan|     overseas inflow|       36|   2|
|            Busan|                 etc|       30|   3|
|            Busan|contact with patient|       19|   4|
|            Busan|  Shincheonji Church|       12|   5|
|            Busan|Haeundae-gu Catho...|        6|   6|
|            Busan|Suyeong-gu Kinder...|        5|   7|
|            Busan|      Jin-gu Academy|        4|   8|
|            Busan|       Itaewon Clubs|        4|   8|
|            Busan|Cheongdo Daenam H...|        1|   9|
|Chungcheongbuk-do|     overseas inflow|       13|   1|
|Chungcheongbuk-do|Goesan-gun Jangye...|       11|   2|
|Chungcheongbuk-do|                 etc|       11|   2|
|Chungcheongbuk-do|       Itaewon Clubs|        9|   3|
|Chungcheongbuk-do|contact with patient|        

In [0]:
def casehighlow(cases):
    if cases < 50:
        return 'low'
    else:
        return 'high'

In [0]:
print(casehighlow(20))

low


In [0]:
caseudf = udf(lambda z: casehighlow(z),StringType())

In [0]:
df.select(col("province") , col("infection_case"), col("confirmed"), caseudf(col("confirmed"))).show()

+--------+--------------------+---------+-------------------+
|province|      infection_case|confirmed|<lambda>(confirmed)|
+--------+--------------------+---------+-------------------+
|   Seoul|       Itaewon Clubs|      139|               high|
|   Seoul|             Richway|      119|               high|
|   Seoul| Guro-gu Call Center|       95|               high|
|   Seoul|Yangcheon Table T...|       43|                low|
|   Seoul|     Day Care Center|       43|                low|
|   Seoul|Manmin Central Ch...|       41|                low|
|   Seoul|SMR Newly Planted...|       36|                low|
|   Seoul|       Dongan Church|       17|                low|
|   Seoul|Coupang Logistics...|       25|                low|
|   Seoul|     Wangsung Church|       30|                low|
|   Seoul|Eunpyeong St. Mar...|       14|                low|
|   Seoul|    Seongdong-gu APT|       13|                low|
|   Seoul|Jongno Community ...|       10|                low|
|   Seou